<h1>Maine Brewery Search: Where is the best town to open a brewery in Maine?<h1>

In [245]:
# importing libraries
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors

import requests 
from pandas.io.json import json_normalize 

from sklearn.cluster import KMeans

!pip install folium
import folium

print('Libraries imported.')

Libraries imported.


In [246]:
# scraping Maine town data from Wikipedia                                                                                                   
url = 'https://en.wikipedia.org/wiki/List_of_towns_in_Maine'

dfs = pd.read_html(url)
                                                                                                        
df = dfs[0]
                                                                                                          
df2 = df[['Town','County']]

print(df2)

                    Town        County
0              Brunswick    Cumberland
1            Scarborough    Cumberland
2                Windham    Cumberland
3                 Gorham    Cumberland
4                   York          York
5               Falmouth    Cumberland
6              Kennebunk          York
7                  Orono     Penobscot
8               Standish    Cumberland
9                  Wells          York
10               Kittery          York
11        Cape Elizabeth    Cumberland
12                Lisbon  Androscoggin
13     Old Orchard Beach          York
14               Topsham     Sagadahoc
15              Yarmouth    Cumberland
16             Skowhegan      Somerset
17              Freeport    Cumberland
18                Buxton          York
19                  Gray    Cumberland
20             Waterboro          York
21            Cumberland    Cumberland
22               Winslow      Kennebec
23               Berwick          York
24            Farmington 

In [247]:
df2.shape

(430, 2)

In [248]:
# scraping Maine city data from Wikipedia                                                                                                   
url = 'https://en.wikipedia.org/wiki/List_of_cities_in_Maine'

dfs2 = pd.read_html(url)[0].rename(columns={'City': 'Town'}, inplace=False)
                                                                                                        
cities_df = dfs2[['Town', 'County']]
cities_df['Town'] = cities_df.Town.str.replace('[^a-zA-Z ]', '')
cities_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Town,County
0,Portland,Cumberland
1,Lewiston,Androscoggin
2,Bangor,Penobscot
3,South Portland,Cumberland
4,Auburn,Androscoggin
5,Biddeford,York
6,Sanford,York
7,Saco,York
8,Augusta,Kennebec
9,Westbrook,Cumberland


In [249]:
cities_df.shape

(23, 2)

In [250]:
towns_df = pd.concat([cities_df, df2])
towns = towns_df['Town'].str.strip()
towns_df['Town'] = towns

In [251]:
towns_df['Town'].unique().shape

(453,)

In [252]:
!pip install requests

In [253]:
# scraping Maine town lat/long from Maps of World    
import requests as rq

url = 'https://www.mapsofworld.com/usa/states/maine/lat-long.html'

res = rq.get(url)
html = res.text

latlongs = pd.read_html(html, header=1)

In [254]:
latlongs_df = latlongs[0].rename(columns={'Location': 'Town'}, inplace=False)
latlongs_df[['Town']] = latlongs_df['Town'].str.replace(' city', '')
latlongs_df[['Town']] = latlongs_df['Town'].str.strip()

In [255]:
latlongs_df

,Town,Latitude,Longitude
0,Anson,44.80,-69.9
1,Ashland,46.62,-68.4
2,Auburn,44.09,-70.25
3,Augusta,44.33,-69.73
4,Bangor,44.83,-68.79
5,Bar Harbor,44.38,-68.21
6,Bath,43.93,-69.83
7,Belfast,44.43,-69.03
8,Berwick,43.27,-70.86
9,Biddeford,43.44,-70.4


In [256]:
# joining for final dataframe  
df_final = pd.merge(towns_df, latlongs_df, how='inner', on='Town')
df_final

,Town,County,Latitude,Longitude
0,Portland,Cumberland,43.65,-70.17
1,Lewiston,Androscoggin,44.09,-70.17
2,Bangor,Penobscot,44.83,-68.79
3,South Portland,Cumberland,43.63,-70.29
4,Auburn,Androscoggin,44.09,-70.25
5,Biddeford,York,43.44,-70.4
6,Sanford,York,43.44,-70.78
7,Saco,York,43.52,-70.42
8,Augusta,Kennebec,44.33,-69.73
9,Westbrook,Cumberland,43.70,-70.35


In [257]:
counties = df_final['County'].unique().tolist()

In [258]:
county_color = {}
for county in counties:
    county_color[county]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [314]:
# create map of Maine towns & counties using latitude and longitude values
map_maine = folium.Map(location=[45.367584, -68.972168], zoom_start=7)

for lat, lng, county, town in zip(df_final['Latitude'], 
                                           df_final['Longitude'],
                                           df_final['County'], 
                                           df_final['Town']):
    label_text = county + ' - ' + town
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=county_color[county],
        fill_color=county_color[county],
        fill_opacity=0.7).add_to(map_maine)  
    
map_maine

In [260]:
CLIENT_ID = 'SZZDW1YDDLYBBNXDZJLLSRKELECP4CX14KQLEJ2RB3FW25VT' 
CLIENT_SECRET = 'MCO2X4IFWSDFH5J0HAHPTW2CZB4K1D1KXH0JCZDRDDSANSTB' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SZZDW1YDDLYBBNXDZJLLSRKELECP4CX14KQLEJ2RB3FW25VT
CLIENT_SECRET:MCO2X4IFWSDFH5J0HAHPTW2CZB4K1D1KXH0JCZDRDDSANSTB


In [293]:
#connect to Foursquare API
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Town', 
                  'Town Latitude', 
                  'Town Longitude',
                  'County',           
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [294]:
maine_venues = getNearbyVenues(names=df_final['Town'],
                                latitudes=df_final['Latitude'],
                                longitudes=df_final['Longitude'])

Portland
Lewiston
Bangor
South Portland
Auburn
Biddeford
Sanford
Saco
Augusta
Westbrook
Waterville
Presque Isle
Brewer
Bath
Caribou
Ellsworth
Old Town
Rockland
Belfast
Gardiner
Calais
Hallowell
Eastport
Brunswick
Scarborough
Gorham
Falmouth
Kennebunk
Orono
Standish
Kittery
Old Orchard Beach
Topsham
Yarmouth
Skowhegan
Freeport
Gray
Winslow
Berwick
Farmington
Hampden
Fairfield
Oakland
Winthrop
Houlton
Rumford
Bridgton
Bar Harbor
Lincoln
Waldoboro
Norway
Bucksport
Camden
North Berwick
Millinocket
Pittsfield
Oxford
Dover-Foxcroft
Naples
Fort Kent
Wilton
Casco
Madawaska
Winterport
Dexter
Wiscasset
Kennebunkport
Richmond
Fryeburg
Fort Fairfield
Norridgewock
Clinton
Newport
Livermore Falls
Milford
Mechanic Falls
Bowdoinham
Farmingdale
Thomaston
Blue Hill
Searsport
Mexico
Dixfield
Anson
Milo
Limestone
Damariscotta
Boothbay Harbor
Machias
Unity
Van Buren
Mapleton
Southwest Harbor
Hartland
Newcastle
Randolph
East Millinocket
Washburn
Greenville
Guilford
Mars Hill
Castine
Lubec
Ashland
Howland
Wo

ValueError: Length mismatch: Expected axis has 7 elements, new values have 8 elements

In [295]:
#count of venues by town
maine_venues.groupby('Town').count()

,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Town,,,,,,
Augusta,2,2,2,2,2,2
Bangor,6,6,6,6,6,6
Bar Harbor,3,3,3,3,3,3
Bath,1,1,1,1,1,1
Berwick,4,4,4,4,4,4
Blue Hill,9,9,9,9,9,9
Boothbay Harbor,3,3,3,3,3,3
Brewer,10,10,10,10,10,10
Brunswick,4,4,4,4,4,4


In [288]:
maine_venues['Venue Category'].unique()[:100]

array(['Brewery', 'American Restaurant', 'Donut Shop', 'Radio Station',
       'Café', 'Music Venue', 'College Theater', 'Bakery', 'Pizza Place',
       'Ice Cream Shop', 'Video Store', 'Automotive Shop', 'Pharmacy',
       'Tanning Salon', 'Spa', 'Coffee Shop', 'Breakfast Spot',
       'Discount Store', 'Sports Bar', 'Campground', 'Trail',
       'Restaurant', 'Boutique', 'Italian Restaurant',
       'Thrift / Vintage Store', 'Indoor Play Area', 'Chinese Restaurant',
       'Gas Station', 'Fast Food Restaurant', 'Mexican Restaurant',
       'Bowling Alley', 'Bar', 'Business Service', 'Seafood Restaurant',
       'Convenience Store', 'Moving Target', 'Thai Restaurant',
       'Bagel Shop', 'Sandwich Place', 'Bank', 'Supermarket',
       'Frozen Yogurt Shop', 'Chiropractor', 'Sporting Goods Shop',
       'Beer Store', 'Athletics & Sports', 'Golf Course', 'Park',
       'Gym / Fitness Center', 'Skate Park', 'Skating Rink',
       'Nature Preserve', 'Market', 'Home Service',
       'Const

In [296]:
to_onehot = pd.get_dummies(maine_venues[['Venue Category']], prefix="", prefix_sep="")

# add town column back to dataframe
to_onehot['Town'] = maine_venues['Town'] 

# move town column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(343, 122)


,Town,Accessories Store,American Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Business Service,Café,Campground,Candy Store,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Coffee Shop,College Theater,Concert Hall,Construction & Landscaping,Convenience Store,Creperie,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,English Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Truck,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Home Service,Hotel,Hunting Supply,Ice Cream Shop,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Lake,Lawyer,Luggage Store,Market,Mediterranean Restaurant,Mexican Restaurant,Motel,Movie Theater,Moving Target,Music Venue,National Park,Nature Preserve,New American Restaurant,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Post Office,Pub,Radio Station,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Tanning Salon,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Video Store,Waterfall,Wine Shop
0,Lewiston,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lewiston,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bangor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bangor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bangor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [297]:
to_grouped = to_onehot.groupby(["Town"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(64, 122)


,Town,Accessories Store,American Restaurant,Athletics & Sports,Automotive Shop,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Business Service,Café,Campground,Candy Store,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Coffee Shop,College Theater,Concert Hall,Construction & Landscaping,Convenience Store,Creperie,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Doctor's Office,Donut Shop,Electronics Store,English Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Truck,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Home Service,Hotel,Hunting Supply,Ice Cream Shop,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Lake,Lawyer,Luggage Store,Market,Mediterranean Restaurant,Mexican Restaurant,Motel,Movie Theater,Moving Target,Music Venue,National Park,Nature Preserve,New American Restaurant,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Post Office,Pub,Radio Station,Restaurant,River,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Tanning Salon,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Video Store,Waterfall,Wine Shop
0,Augusta,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.000000,0.000000,0.00,0.000000,0.000000,0.500000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bangor,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.333333,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bar Harbor,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.

In [298]:
len(to_grouped[to_grouped["Brewery"] > 0])

9

In [300]:
to_brew = to_grouped[["Town","Brewery"]]

In [301]:
to_brew.head()

,Town,Brewery
0,Augusta,0.0
1,Bangor,0.0
2,Bar Harbor,0.0
3,Bath,0.0
4,Berwick,0.0


In [302]:
from sklearn.cluster import KMeans
toclusters = 5

to_clustering = to_brew.drop(["Town"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 3], dtype=int32)

In [303]:
to_merged = to_brew.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [304]:
to_merged.rename(columns={"Town": "Town"}, inplace=True)
to_merged.head(5)

,Town,Brewery,Cluster Labels
0,Augusta,0.0,0
1,Bangor,0.0,0
2,Bar Harbor,0.0,0
3,Bath,0.0,0
4,Berwick,0.0,0


In [305]:
to_merged = to_merged.join(maine_venues.set_index("Town"), on="Town")

print(to_merged.shape)
to_merged.head()

(343, 9)


,Town,Brewery,Cluster Labels,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Augusta,0.0,0,44.33,-69.73,Twins Country Store,44.329215,-69.732778,Restaurant
0,Augusta,0.0,0,44.33,-69.73,Rideouts Pit Stop,44.329071,-69.732697,Pizza Place
1,Bangor,0.0,0,44.83,-68.79,Dunkin',44.834229,-68.790280,Donut Shop
1,Bangor,0.0,0,44.83,-68.79,WHSN 89.3FM,44.829003,-68.792933,Radio Station
1,Bangor,0.0,0,44.83,-68.79,Wileys Cafe Husson,44.827049,-68.792063,Café


In [282]:
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged.head()

,Town,Brewery,Cluster Labels,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Augusta,0.0,0,44.33,-69.73,Twins Country Store,44.329215,-69.732778,Restaurant
28,Kennebunk,0.0,0,43.38,-70.54,Crossfit KGB,43.382061,-70.538879,Gym / Fitness Center
28,Kennebunk,0.0,0,43.38,-70.54,Rogers Park,43.380721,-70.541375,Park
28,Kennebunk,0.0,0,43.38,-70.54,Squaretoes,43.382132,-70.545449,Café
27,Hampden,0.0,0,44.75,-68.84,"RT 202 & Coldbrook Rd, Hampden",44.754455,-68.839624,Intersection


In [315]:
# create map
map_clusters = folium.Map(location=[45.367584, -68.972168], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i + x + (i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Town Latitude'], to_merged['Town Longitude'], to_merged['Cluster Labels'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [306]:
#Cluster 0
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'Brewery') ]

,Town,Brewery,Cluster Labels,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [307]:
#Cluster 1
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['Venue Category'] == 'Brewery') ]

,Town,Brewery,Cluster Labels,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
31,Lewiston,0.5,1,44.09,-70.17,Brentwood Beer Company,44.086422,-70.169558,Brewery


In [308]:
#Cluster 2
to_merged.loc[(to_merged['Cluster Labels'] ==2) & (to_merged['Venue Category'] == 'Brewery') ]

,Town,Brewery,Cluster Labels,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,Blue Hill,0.111111,2,44.41,-68.59,Deep Water Brewing,44.411596,-68.592627,Brewery
30,Kittery,0.142857,2,43.09,-70.74,Tributary Brewing Company,43.090371,-70.735942,Brewery
34,Lubec,0.125000,2,44.86,-66.99,Lubec Brewing Company,44.860983,-66.983813,Brewery
51,South Portland,0.111111,2,43.63,-70.29,Fore River Brewing Co.,43.632717,-70.287773,Brewery


In [309]:
#Cluster 3
to_merged.loc[(to_merged['Cluster Labels'] ==3) & (to_merged['Venue Category'] == 'Brewery') ]

,Town,Brewery,Cluster Labels,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
9,Camden,0.037037,3,44.21,-69.07,Sea Dog Brewing Company,44.209864,-69.064759,Brewery
23,Gorham,0.041667,3,43.68,-70.44,Sebago Brewing Company,43.678577,-70.439264,Brewery


In [310]:
#Cluster 4
to_merged.loc[(to_merged['Cluster Labels'] ==4) & (to_merged['Venue Category'] == 'Brewery') ]

,Town,Brewery,Cluster Labels,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
18,Farmington,0.090909,4,44.67,-70.15,The Roost,44.669910,-70.150809,Brewery
63,Yarmouth,0.071429,4,43.80,-70.19,Brickyard Hollow Brewing,43.800921,-70.188372,Brewery
